In [1]:
from __future__ import print_function

import os
import random
import shutil

import skimage as ski
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


import numpy as np

from hyperopt import Trials, STATUS_OK, tpe
from keras.datasets import mnist
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from keras.models import Sequential
from keras.utils import np_utils

from hyperas import optim
from hyperas.distributions import choice, uniform

#=============================================================================================
# Precheck on directory structure
#=============================================================================================

if os.listdir('test_data/cell/'):
    print('Warning! Files detected in test data directory!')
    print('Moving files back to training data directory...')

    for f in os.listdir('test_data/cell/'):
        shutil.move('test_data/cell/'+f,'training_data/cell/'+f)

    for f in os.listdir('test_data/nocell/'):
        shutil.move('test_data/nocell/'+f,'training_data/nocell/'+f)
        
    print('Done!\n')

/opt/anaconda/envs/deeplearn/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Warning! Files detected in test data directory!
Moving files back to training data directory...
Done!



In [2]:
cell_data = os.listdir('training_data/cell/')
nocell_data = os.listdir('training_data/nocell/')

validation_cell_data = random.sample(cell_data, int(0.3*len(cell_data)))
validation_nocell_data = random.sample(nocell_data, int(0.3*len(nocell_data)))

for f in validation_cell_data:
    shutil.move('training_data/cell/'+f,'test_data/cell/'+f)

for f in validation_nocell_data:
    shutil.move('training_data/nocell/'+f,'test_data/nocell/'+f)

print('Done!\n')

Done!



In [3]:
def data():
    """
    Data providing function:

    This function is separated from create_model() so that hyperopt
    won't reload data for each evaluation run.
    """  
    cell_data = os.listdir('training_data/cell/')
    nocell_data = os.listdir('training_data/nocell/')

    # Training data
    path = 'training_data/cell/'
    x_train = []
    y_train = []
    for f in cell_data:
        im = mpimg.imread(str(path)+f)
        x_train.append(im)
        y_train.append(1)
    path = 'training_data/nocell/'
    for f in nocell_data:
        im = mpimg.imread(str(path)+f)
        x_train.append(im)
        y_train.append(0)

    cell_test_data = os.listdir('test_data/cell/')
    nocell_test_data = os.listdir('test_data/nocell/')

    # Validation data
    path = 'test_data/cell/'
    x_test = []
    y_test = []
    for f in cell_test_data:
        im = mpimg.imread(str(path)+f)
        x_test.append(im)
        y_test.append(1)
    path = 'test_data/nocell/'
    for f in nocell_test_data:
        im = mpimg.imread(str(path)+f)
        x_test.append(im)
        y_test.append(0)
        
    
    x_train = np.asarray(x_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.float32)
    x_test = np.asarray(x_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.float32)

    x_train = x_train.reshape(x_train.shape[0], 80, 80, 1)
    x_test = x_test.reshape(x_test.shape[0], 80, 80, 1)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    nb_classes = 2
    y_train = np_utils.to_categorical(y_train, nb_classes)
    y_test = np_utils.to_categorical(y_test, nb_classes)
    return x_train, y_train, x_test, y_test


def create_model(x_train, y_train, x_test, y_test):
    """
    Model providing function:

    Create Keras model with double curly brackets dropped-in as needed.
    Return value has to be a valid python dictionary with two customary keys:
        - loss: Specify a numeric evaluation metric to be minimized
        - status: Just use STATUS_OK and see hyperopt documentation if not feasible
    The last one is optional, though recommended, namely:
        - model: specify the model just created so that we can later use it again.
    """
    model = Sequential()
    
    model_choice = {{choice(['one', 'two', 'three'])}}
    if model_choice == 'one':
        model.add(Conv2D(16, kernel_size=3, activation='relu',padding='same', input_shape=(1,28,28), data_format='channels_first'))
        model.add(Conv2D(16, kernel_size=3, activation='relu',padding='same'))
        model.add(MaxPooling2D(pool_size=2,strides=2))
        model.add(Dropout({{uniform(0, 1)}}))
        model.add(Conv2D(32, kernel_size=3, activation='relu'))
        model.add(Conv2D(32, kernel_size=3, activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=2,strides=2))
        model.add(Dropout({{uniform(0, 1)}}))
    elif model_choice == 'two':
        model.add(Conv2D(32, kernel_size=3, activation='relu',padding='same', input_shape=(1,28,28), data_format='channels_first'))
        model.add(Conv2D(32, kernel_size=3, activation='relu',padding='same'))
        model.add(MaxPooling2D(pool_size=2,strides=2))
        model.add(Dropout({{uniform(0, 1)}}))
        model.add(Conv2D(64, kernel_size=3, activation='relu'))
        model.add(Conv2D(64, kernel_size=3, activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=2,strides=2))
        model.add(Dropout({{uniform(0, 1)}}))
# Below is Gerald's original model
    elif model_choice == 'three':
        model.add(Conv2D(8, (3, 3), input_shape = (80, 80, 1), activation = 'relu'))
        model.add(MaxPooling2D(pool_size = (2, 2)))
        model.add(Conv2D(16, (3, 3), input_shape = (80, 80, 1), activation = 'relu'))
        model.add(MaxPooling2D(pool_size = (2, 2)))
        model.add(BatchNormalization())
        model.add(Conv2D(32, (3, 3), input_shape = (80, 80, 1), activation = 'relu'))
        model.add(MaxPooling2D(pool_size = (2, 2)))
        model.add(BatchNormalization())
        model.add(Conv2D(32, (3, 3), input_shape = (80, 80, 1), activation = 'relu'))
        model.add(MaxPooling2D(pool_size = (2, 2)))
        model.add(BatchNormalization())
        model.add(Dropout({{uniform(0, 1)}}))
        model.add(Flatten())
        model.add(Dense({{choice([128, 256, 512])}}))
        model.add(Activation({{choice(['relu', 'sigmoid'])}}))
        model.add(Dropout({{uniform(0, 1)}}))
    
    model.add(Dense({{choice([256, 512, 1024])}}))
    model.add(Activation({{choice(['relu', 'sigmoid'])}}))
    model.add(Dropout({{uniform(0, 1)}}))

    model.add(Flatten())
    model.add(Dense(2, activation='softmax'))
    
    model.summary()

    model.compile(loss='binary_crossentropy', metrics=['accuracy'],
                  optimizer={{choice(['rmsprop', 'adam', 'sgd'])}})

    result = model.fit(x_train, y_train,
              batch_size={{choice([64, 128])}},
              epochs=2,
              verbose=2,
              validation_split=0.1)
    #get the highest validation accuracy of the training epochs
    validation_acc = np.amax(result.history['val_acc']) 
    print('Best validation acc of epoch:', validation_acc)
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}


if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=5,
                                          trials=Trials(),
                                          notebook_name='hyperas')
    X_train, Y_train, X_test, Y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, Y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import os
except:
    pass

try:
    import random
except:
    pass

try:
    import shutil
except:
    pass

try:
    import skimage as ski
except:
    pass

try:
    import matplotlib.pyplot as plt
except:
    pass

try:
    import matplotlib.image as mpimg
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from keras.datasets import mnist
except:
    pass

try:
    from keras.layers.core import Dense, Dropout, Activation, Flatten
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.utils import np_utils
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

>>> Hyperas search space:

def get_space():
    return {
        'model_choice': hp.choice('model_choice', ['one', 'two', '

NameError: global name 'Conv2D' is not defined

cell_data = os.listdir('training_data/cell/')
nocell_data = os.listdir('training_data/nocell/')

# Training data
path = 'training_data/cell/'
x_train = []
y_train = []
for f in cell_data:
    im = mpimg.imread(str(path)+f)
    x_train.append(im)
    y_train.append(1)
path = 'training_data/nocell/'
for f in nocell_data:
    im = mpimg.imread(str(path)+f)
    x_train.append(im)
    y_train.append(0)
    
cell_test_data = os.listdir('test_data/cell/')
nocell_test_data = os.listdir('test_data/nocell/')
    
# Validation data
path = 'test_data/cell/'
x_test = []
y_test = []
for f in cell_test_data:
    im = mpimg.imread(str(path)+f)
    x_test.append(im)
    y_test.append(1)
path = 'test_data/nocell/'
for f in nocell_test_data:
    im = mpimg.imread(str(path)+f)
    x_test.append(im)
    y_test.append(0)
    
print(x_train, y_train, x_test, y_test) 